In [38]:
# general
import numpy as np
import pandas as pd
from IPython.display import display
import importlib

# first used in exercise one
import kernelsvm as svm
from sklearn.metrics.pairwise import linear_kernel, polynomial_kernel
from sklearn.datasets import fetch_mldata
from sklearn import preprocessing
from sklearn.model_selection import train_test_split

In [12]:
importlib.reload(svm)

<module 'kernelsvm' from '/Users/andrewenfield/work/github/Data558/Week09/kernelsvm.py'>

In [2]:
x_simple = np.array([3,2,0,1,-1,-2]).reshape(3,2)
x_simple

array([[ 3,  2],
       [ 0,  1],
       [-1, -2]])

Note: Per the request in the "Collaboration policy" note, I've discussed at least part of this assignment with many of the MS employees in the class, including Abhishek, Geoff, Suman, and Charles. (Different weeks/different assignments have different people, depending upon who attends our study groups, but I'll probably just include this blurb w/ each homework since it's generally correct.) I've also gotten input from the discussion board.

# Exercise one

_Compute the gradient ∇F(α) of F.

_Write a function computegram that computes, for any set of datapoints x1, . . . , xn, the kernel matrix K._

I implemented the computegram_linear function, and all of the supporting functions including the gradient and objective functions, in the file kernelsvm.py, which I imported into this notebook with the alias svm. I also include my (relatively minimal) unit tests, in linearsvm-test.py.

In [10]:
svm.computegram_linear(x_simple)

array([[13,  2, -7],
       [ 2,  1, -2],
       [-7, -2,  5]])

In [11]:
svm.computegram_polynomial(x_simple, 2, 1)

array([[196,   9,  36],
       [  9,   4,   1],
       [ 36,   1,  36]])

_Write a function kerneleval that computes, for any set of datapoints x1, . . . , xn and a new datapoint x⋆, the vector of kernel evaluations [k(x1, x⋆), . . . , k(xn, x⋆)]T._

My understanding here is that I'm not using the kernel matrix - that is, the reference to 'kernel evaluation' means to use the kernel _function_, not the kernel matrix. I think this is just (part of - since no alpha weighting yet) the predict step, which - for a linear kernel - is the sum of the inner products between each observation and the value to be predicted, weighted by each learned value of alpha.

In [17]:
svm.kerneleval_linear(x_simple, np.array([2,2]))

array([10,  2, -6])

**TODO** Is there a way this function should be generalized to work with different kinds of kernels?

_Consider the MNIST dataset. You can find instructions on how to download it here: http://scikit-learn.org/stable/datasets/mldata.html. Pick two classes of your choice. Your are going to work on the dataset consisting of the data from these two classes. Standardize the data, if you have not done so already._

In [26]:
mnist = fetch_mldata('MNIST original')
mnist.data.shape, mnist.target.shape

((70000, 784), (70000,))

We'll use 1 and 8. 

In [32]:
ones_and_eights = (mnist.target == 1) | (mnist.target == 8)
X = mnist.data[ones_and_eights]
y = mnist.target[ones_and_eights]
X.shape, y.shape, np.unique(y)

((14702, 784), (14702,), array([ 1.,  8.]))

In [34]:
X_scaled = preprocessing.scale(X)
X_scaled.shape

/Users/andrewenfield/miniconda3/envs/data558/lib/python3.6/site-packages/sklearn/utils/validation.py:429: DataConversionWarning: Data with input dtype uint8 was converted to float64 by the scale function.
  warnings.warn(msg, _DataConversionWarning)


(14702, 784)

In [40]:
X_scaled_test, X_scaled_train, y_test, y_train = train_test_split(X_scaled, y)
X_scaled_test.shape, X_scaled_train.shape, y_test.shape, y_train.shape

((11026, 784), (3676, 784), (11026,), (3676,))

_Write a function mysvm that implements the fast gradient algorithm to train the kernel support vector machine with the squared hinge loss. The function takes as input the initial step-size value for the backtracking rule and a maximum number of iterations._

As noted above, I implemented this in the kernelsvm.py file. Instead of creating a function named mysvm, I just call my fastgradalgo function and pass in references to the kernel SVM gradient and objective functions (which are also implemented in the same file).

_Train your kernel support vector machine with the squared hinge loss and the polyno- mial kernel of order 7 on the the MNIST dataset, tuning the regularization parameter λ using cross-validation._

_Compare the performance of kernel SVMs with different kernels (polynomial kernels with different orders, Gaussian RBF with different bandwidths, etc.)._